In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=BblV6AQsd2s&ab_channel=RiseAgainstVEVO"

# Setting up the model
Define the LLM model to use

In [7]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

model.invoke("Are we in a simulation?")

AIMessage(content='The idea that we might be living in a simulation, similar to the concept portrayed in movies like The Matrix, is a topic of debate among scientists and philosophers. Some argue that advancements in technology and the possibility of creating highly realistic virtual worlds suggest that it is theoretically possible for us to be living in a simulated reality. However, there is currently no definitive evidence to prove or disprove this theory. Ultimately, the question of whether we are in a simulation remains a speculative and philosophical one.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

# Using the magic of Chains :)

In [8]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# Chain example
chain = model | parser

chain.invoke("Are we in a simulation?")

'This is a philosophical question that has been debated by many thinkers. Some believe that it is possible that we are living in a simulation created by a more advanced civilization or technology, while others argue that there is no evidence to support this theory. Ultimately, it is a question that may never have a definitive answer.'

# Using prompt templates

In [9]:
from langchain_core.prompts import ChatPromptTemplate

template = """
  Answer the question based on the context below. If you can't
  answer the question, just say "I don't know".

  Context: {context}

  Question: {question}
  """

prompt = ChatPromptTemplate.from_template(template)
print(prompt.format(context="The sky is blue.", question="What color is the sky?"))

print("Did it work?")

chain = prompt | model | parser
chain.invoke({
  "context": "I have an appointment tomorrow at 3pm.", 
  "question": "Can I go for lunch tomorrow at 3:00PM?"
})

Human: 
  Answer the question based on the context below. If you can't
  answer the question, just say "I don't know".

  Context: The sky is blue.

  Question: What color is the sky?
  
Did it work?


'No, you have an appointment at 3pm tomorrow.'

# Combining Chains

In [10]:
from operator import itemgetter

translation_template = """
  Translate the 

  answer: {answer}

  to

  language: {language}
  """

translation_prompt = ChatPromptTemplate.from_template(translation_template)

translation_chain = (
  {"answer": chain, "language": itemgetter("language") } | translation_prompt | model | parser
)

translation_chain.invoke({
  "context": "I have an appointment tomorrow at 3pm.", 
  "question": "Can I go for lunch tomorrow at 3:00PM?",
  "language": "es"
})


'Respuesta: No, tienes una cita a las 3pm mañana, por lo que no puedes ir a almorzar a esa hora.'